## ouster 数据处理
Kitti标签说明：https://www.yuque.com/huangzhongqing/hre6tf/qcx4vz#y64Af
3D bounding box: (label，cx, cy, cz, dx, dy, dz, heading)
其中，
*  label：目标类别，一共6类，具体信息见下面【标注label】内容 （Truck，Car，Pedestrian，Excavator，Widebody，Auxiliary）
    *  ['Truck'，'Car'，'Pedestrian'，'Excavator'，'Widebody'，'Auxiliary（Others）']
*  (cx, cy, cz) ：框的中心点，即为物体3D框的几何中心位置
*  (dx, dy, dz)：框的长宽高。具体为物体3D框在heading角度为0时沿着x-y-z三个方向的长度
*  heading：为物体在俯视图下的朝向角 (沿着x轴方向为0度角，逆时针x到y角度增加)
```
# 一行数据
Truck 32.11975965338263 -45.413255270899725 -0.37482731425800486 8.478923809647586 6.534787325995467 5.85106455969887 2.521077687071955
```
参考：/home/hcq/pointcloud/mmdetection3d/tools/data_converter/kitti_data_utils.py

In [7]:

# Ouster GT数据===========================================================
import numpy as np
def get_label_anno_ouster(label_path): # 'data/kitti/training/label_2/000010.txt'
    annotations = {}
    annotations.update({
        'name': [], # 名字
        # 'truncated': [],
        # 'occluded': [],
        # 'alpha': [],
        # 'bbox': [], # 2D
        'location': [], # 中心点坐标
        'dimensions': [], # 需要 长宽高
        'rotation_y': []
    })
    with open(label_path, 'r') as f: # 
        lines = f.readlines() # 打开文件
    # if len(lines) == 0 or len(lines[0]) < 15:
    #     content = []
    # else:
    content = [line.strip().split(' ') for line in lines] # GT内容
    print(content)
    num_objects = len([x[0] for x in content if x[0] != 'DontCare']) # 除了Dontare，有多少类，这里是2类
    annotations['name'] = np.array([x[0] for x in content]) # 第一列：['Car' 'Van' 'DontCare' 'DontCare' 'DontCare']
    num_gt = len(annotations['name'])
    annotations['location'] = np.array([[float(info) for info in x[1:4]] # 中心xyz坐标
                                        for x in content]).reshape(-1, 3)
    annotations['dimensions'] = np.array([[float(info) for info in x[4:7]] #  表示该车的高度，宽度，和长度，单位为米。（H,W,L）
                                          for x in content
                                          ]).reshape(-1, 3)[:, [0, 2, 1]] # 长宽高位置--> 长度 高度，宽度？？？？
    annotations['rotation_y'] = np.array([float(x[7]) # 表示车体朝向，绕相机坐标系y轴的弧度值
                                          for x in content]).reshape(-1)
    # 如果有第9列置信度
    print(len(content[0]))
    if len(content) != 0 and len(content[0]) == 9:  # have score #  (预测有score，但label_2标签文件不包含score)
        annotations['score'] = np.array([float(x[8]) for x in content])
    else:
        annotations['score'] = np.zeros((annotations['rotation_y'].shape[0], ))
    index = list(range(num_objects)) + [-1] * (num_gt - num_objects) # [0, 1, -1, -1, -1] = [0,1 ] +  [-1, -1, -1]
    annotations['index'] = np.array(index, dtype=np.int32)
    annotations['group_ids'] = np.arange(num_gt, dtype=np.int32) # [0 1 2 3 4]
    return annotations


In [8]:
str = "/home/hcq/pointcloud/Python/data_process/txt/txt_kitti/000000ouster.txt"
ann = get_label_anno_ouster(str)
ann

[['Truck', '23.993548145171815', '-12.739941812507684', '-0.36031911422858276', '12.504828276858674', '8.34002923274409', '7.532050105135633', '-0.9437256977598043'], ['Excavator', '46.18895309993388', '-44.8425769813457', '-0.18315541744232178', '16.711626142283873', '3.9203361646195454', '6.98018099784851', '0.531891184075941'], ['Widebody', '55.29291027745209', '34.22592295080128', '-0.6067225597216265', '4.042755973781112', '1.658953235199069', '4.899275016370133', '0.24802047385311177'], ['Truck', '65.79371238977741', '16.640755268172832', '2.6541123776025857', '2.647803534385339', '6.272589396148364', '9.575877608381253', '1.43664689019158'], ['Pedestrian', '13.80351455283122', '-8.356476903250732', '-2.887029528617859', '0.3560365608098151', '0.23894134393921918', '1.6401670169830322', '0.4992040685384871'], ['Pedestrian', '14.937343649598848', '-7.02851920158248', '-2.79945002445566', '0.5431902327449888', '0.4585907118794559', '1.7716600777504312', '0.5334779995021175'], ['Car

{'name': array(['Truck', 'Excavator', 'Widebody', 'Truck', 'Pedestrian',
        'Pedestrian', 'Car', 'Truck'], dtype='<U10'),
 'location': array([[ 23.99354815, -12.73994181,  -0.36031911],
        [ 46.1889531 , -44.84257698,  -0.18315542],
        [ 55.29291028,  34.22592295,  -0.60672256],
        [ 65.79371239,  16.64075527,   2.65411238],
        [ 13.80351455,  -8.3564769 ,  -2.88702953],
        [ 14.93734365,  -7.0285192 ,  -2.79945002],
        [ 21.95968712, -47.09619542,  -3.63470111],
        [ 32.11975965, -45.41325527,  -0.37482731]]),
 'dimensions': array([[ 7.53205011, 12.50482828,  8.34002923],
        [ 6.980181  , 16.71162614,  3.92033616],
        [ 4.89927502,  4.04275597,  1.65895324],
        [ 9.57587761,  2.64780353,  6.2725894 ],
        [ 1.64016702,  0.35603656,  0.23894134],
        [ 1.77166008,  0.54319023,  0.45859071],
        [ 1.5299761 ,  4.47251824,  0.45812278],
        [ 5.85106456,  8.47892381,  6.53478733]]),
 'rotation_y': array([-0.9437257 , 

## 得到gt_bboxes_3d和gt_labels_3d
*  调用 get_anno_info() ，加载 anno 里面的 boxes， 格式为 (x_lidar, y_lidar, z_lidar, dx, dy, dz, yaw)

参考：/home/hcq/pointcloud/mmdetection3d/mmdet3d/datasets/ouster_dataset.py

In [4]:
# 需要修改gt_bboxes_3d和gt_labels_3d=====================================
import copy
def get_ann_info(ann):
    """Get annotation info according to the given index.

    Args:
        index (int): Index of the annotation data to get.

    Returns:
        dict: annotation information consists of the following keys:

            - gt_bboxes_3d (:obj:`LiDARInstance3DBoxes`): \
                3D ground truth bboxes.
            - gt_labels_3d (np.ndarray): Labels of ground truths.====================================================================
            - gt_bboxes (np.ndarray): 2D ground truth bboxes.
            - gt_labels (np.ndarray): Labels of ground truths.
            - gt_names (list[str]): Class names of ground truths.
    """
    # Use index to get the annos, thus the evalhook could also use this api
    info = ann
    # rect = info['calib']['R0_rect'].astype(np.float32)
    # Trv2c = info['calib']['Tr_velo_to_cam'].astype(np.float32)

    annos = info
    # we need other objects to avoid collision when sample
    # annos = self.remove_dontcare(annos)
    loc = annos['location']
    dims = annos['dimensions']
    rots = annos['rotation_y']
    gt_names = annos['name']
    gt_bboxes_3d = np.concatenate([loc, dims, rots[..., np.newaxis]],
                                    axis=1).astype(np.float32)
    # 修改
    # gt_bboxes_3d = LiDARInstance3DBoxes(gt_bboxes_3d) # ====================================================================
    # convert gt_bboxes_3d to velodyne coordinates  格式为 (x_lidar, y_lidar, z_lidar, dx, dy, dz, yaw)
    # gt_bboxes_3d = CameraInstance3DBoxes(gt_bboxes_3d).convert_to(
    #     self.box_mode_3d, np.linalg.inv(rect @ Trv2c))
    # gt_bboxes = annos['bbox']

    # selected = self.drop_arrays_by_name(gt_names, ['DontCare']) # 不要DonCare
    # gt_bboxes = gt_bboxes[selected].astype('float32')
    # gt_names = gt_names[selected]
    CLASSES =  ('Truck','Car','Pedestrian','Excavator','Widebody','Auxiliary')
    gt_labels_3d = []
    for cat in gt_names: # 变成数字的形式
        if cat in CLASSES:
            gt_labels_3d.append(CLASSES.index(cat))
        else:
            gt_labels_3d.append(-1)
    gt_labels_3d = np.array(gt_labels_3d).astype(np.int64)

    anns_results = dict(
        gt_bboxes_3d=gt_bboxes_3d,
        gt_labels_3d=gt_labels_3d,
        # bboxes=gt_bboxes,
        # labels=gt_labels,
        gt_names=gt_names)
    return anns_results

In [24]:
ann_info = get_ann_info(ann)
ann_info

{'gt_bboxes_3d': array([[ 23.993547  , -12.739942  ,  -0.3603191 ,  12.504828  ,
           8.340029  ,   7.53205   ,  -0.9437257 ],
        [ 46.188953  , -44.84258   ,  -0.18315542,  16.711626  ,
           3.9203362 ,   6.980181  ,   0.53189117],
        [ 55.29291   ,  34.22592   ,  -0.60672253,   4.042756  ,
           1.6589532 ,   4.899275  ,   0.24802047],
        [ 65.79371   ,  16.640755  ,   2.6541123 ,   2.6478035 ,
           6.272589  ,   9.575877  ,   1.4366469 ],
        [ 13.8035145 ,  -8.356477  ,  -2.8870296 ,   0.35603657,
           0.23894134,   1.640167  ,   0.49920407],
        [ 14.937344  ,  -7.028519  ,  -2.79945   ,   0.54319024,
           0.45859072,   1.7716601 ,   0.533478  ],
        [ 21.959686  , -47.096195  ,  -3.634701  ,   4.4725184 ,
           0.4581228 ,   1.5299761 ,   1.1090727 ],
        [ 32.11976   , -45.413254  ,  -0.37482733,   8.478924  ,
           6.534787  ,   5.8510647 ,   2.5210776 ]], dtype=float32),
 'gt_labels_3d': array([0, 3, 4